In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt_kwargs = {'figsize': (10, 4)}
from IPython.display import HTML

First, we will define eco rating with the formula $$EcoRatingRound(LO_A, LO_B, Win_A)=\begin{cases}\alpha LO_B/LO_A&\text{if}\ Win_A = 1\\
                                                                                                \beta LO_A/LO_B& \text{if}\ Win_A = 0                       
\end{cases}$$

EcoRating of team A in a match will be the average over all the EcoRatingRounds. (LO stands for loadout value)

In [99]:
alpha = 1 #Eco rating parameters
beta = -1

In [100]:
#Rating for a round: winning with low credits earns more rating than winning with high credits, losing with more credits earns more negative rating.
def EcoRoundRating(LoadOutA, LoadoutB,  OutcomeA):
    LoadoutB = max(LoadoutB, 100)   #This to avoid any division by 0 errors
    LoadOutA = max(LoadOutA, 100)   #
    if OutcomeA == 1:
        return alpha*LoadoutB/LoadOutA
    if OutcomeA == 0:
        return beta*LoadOutA/LoadoutB
    
#Another rating system which gives more importance to rounds with similar LO from both teams.
def EcoRoundRating1(LoadOutA, LoadoutB, Eco_type_A, Eco_type_B,  OutcomeA):
    LoadoutB = max(LoadoutB, 100)
    LoadOutA = max(LoadOutA, 100)
    if Eco_type_A == Eco_type_B:
        if OutcomeA == 1:
            return alpha*LoadoutB/LoadOutA
        if OutcomeA == 0:
            return beta*LoadOutA/LoadoutB
    if Eco_type_A != Eco_type_B:
        if OutcomeA == 1:
            return LoadoutB/LoadOutA
        if OutcomeA == 0:
            return LoadOutA/LoadoutB
    

In [101]:
#players_stats = pd.read_csv("../data/vct_2023/players_stats/players_stats.csv")
#overview = pd.read_csv("../data/vct_2023/matches/overview.csv")
maps_scores = pd.read_csv("../data/vct_2023/matches/maps_scores.csv")
eco_rounds = pd.read_csv("../data/vct_2023/matches/eco_rounds.csv")

In [102]:
maps_scores["Team_A_win"] = maps_scores["Team A Score"] > maps_scores["Team B Score"]
maps_scores["Team_B_win"] = maps_scores["Team B Score"] > maps_scores["Team A Score"]

maps_scores = maps_scores[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team_A_win", "Team B", "Team_B_win"]]

In [103]:
len(eco_rounds[eco_rounds["Loadout Value"].isna()])

0

In [104]:
eco_rounds["Outcome"] = eco_rounds["Outcome"].apply(lambda x: 1 if x == "Win" else 0)
eco_rounds["Loadout Value"] = eco_rounds["Loadout Value"].apply(lambda x: int(x.replace(".","").replace("k","00")))
eco_rounds["Remaining Credits"] = eco_rounds["Remaining Credits"].apply(lambda x: int(x.replace(".","").replace("k","00")))

#We droped this since we didn't use these columns
eco_rounds=eco_rounds.drop(columns=["Type", "Remaining Credits"])

eco_rounds.head(10)

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Team,Loadout Value,Outcome
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,1,Team Liquid,3600,1
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,1,Natus Vincere,3500,0
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,2,Team Liquid,15600,1
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,2,Natus Vincere,4200,0
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,3,Team Liquid,13300,1
5,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,3,Natus Vincere,18200,0
6,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,4,Team Liquid,21400,1
7,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,4,Natus Vincere,7400,0
8,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,5,Team Liquid,23600,0
9,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,5,Natus Vincere,20300,1


In [105]:
team_ab_2023 = maps_scores[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team B"]]

team_ab_2023

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team B
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Team Liquid,Natus Vincere
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Bind,Team Liquid,Natus Vincere
2,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Lotus,DRX,LOUD
3,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Split,DRX,LOUD
4,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Ascent,DRX,LOUD
...,...,...,...,...,...,...,...
825,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,LOUD,FNATIC
826,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,LOUD,FNATIC
827,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,LOUD,FNATIC
828,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,LOUD,FNATIC


In [106]:
eco_rounds.drop(columns=["Round Number", "Loadout Value", "Outcome"]).drop_duplicates().shape

(1542, 6)

In [107]:
1542/2

771.0

Eco Rounds has fewer data than map scores. 

Next, we define a dataframe with Loadout value of team A and team B as columns.

In [108]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team']

Eco_rounds_ab_2023=\
pd.merge(
    maps_scores.rename(columns={"Team A":"Team"})\
    ,eco_rounds\
    ,on=keys
)\
.rename(columns={"Loadout Value":"Loadout_A", "Remaining Credits":"Remaining_credit_A", "Type":"Eco_type_A", "Outcome":"Outcome_A"})\
.rename(columns={"Team":"Team A", "Team B":"Team"})\
.set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team', 'Round Number'])\
.join(eco_rounds\
      .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team', 'Round Number'])\
)\
.reset_index()\
.rename(columns={"Loadout Value":"Loadout_B", "Remaining Credits":"Remaining_credit_B", "Type":"Eco_type_B", "Outcome":"Outcome_B"})\
.rename(columns={"Team":"Team B"})

Eco_rounds_ab_2023.head()

,Tournament,Stage,Match Type,Match Name,Map,Team B,Round Number,Team A,Team_A_win,Team_B_win,Loadout_A,Outcome_A,Loadout_B,Outcome_B
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,1,Team Liquid,False,True,3600,1,3500,0
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,2,Team Liquid,False,True,15600,1,4200,0
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,3,Team Liquid,False,True,13300,1,18200,0
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,4,Team Liquid,False,True,21400,1,7400,0
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,5,Team Liquid,False,True,23600,0,20300,1


Apply the rating formula

In [109]:
Eco_rounds_ab_2023["Eco_Round_Rating_A"]=Eco_rounds_ab_2023[["Loadout_A", "Loadout_B", "Outcome_A"]]\
    .apply(lambda row: EcoRoundRating(row["Loadout_A"], row["Loadout_B"], row["Outcome_A"]), axis=1)

Eco_rounds_ab_2023["Eco_Round_Rating_B"]=Eco_rounds_ab_2023[["Loadout_A", "Loadout_B", "Outcome_B"]]\
    .apply(lambda row: EcoRoundRating(row["Loadout_B"], row["Loadout_A"], row["Outcome_B"]), axis=1)


Eco_rounds_ab_2023.head(10)

,Tournament,Stage,Match Type,Match Name,Map,Team B,Round Number,Team A,Team_A_win,Team_B_win,Loadout_A,Outcome_A,Loadout_B,Outcome_B,Eco_Round_Rating_A,Eco_Round_Rating_B
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,1,Team Liquid,False,True,3600,1,3500,0,0.972222,-0.972222
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,2,Team Liquid,False,True,15600,1,4200,0,0.269231,-0.269231
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,3,Team Liquid,False,True,13300,1,18200,0,1.368421,-1.368421
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,4,Team Liquid,False,True,21400,1,7400,0,0.345794,-0.345794
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,5,Team Liquid,False,True,23600,0,20300,1,-1.162562,1.162562
5,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,6,Team Liquid,False,True,22200,1,21300,0,0.959459,-0.959459
6,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,7,Team Liquid,False,True,22000,0,19700,1,-1.116751,1.116751
7,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,8,Team Liquid,False,True,19800,0,19700,1,-1.005076,1.005076
8,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,9,Team Liquid,False,True,19000,0,21100,1,-0.900474,0.900474
9,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Natus Vincere,10,Team Liquid,False,True,19000,1,22900,0,1.205263,-1.205263


Calculate match EcoRating by taking the average.

In [110]:
grouped_eco_rating_df_A = Eco_rounds_ab_2023.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])[["Eco_Round_Rating_A", "Team_A_win"]].agg("mean").reset_index()
grouped_eco_rating_df_B = Eco_rounds_ab_2023.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])[["Eco_Round_Rating_B", "Team_B_win"]].agg("mean").reset_index()


In [111]:
grouped_eco_rating_df_A

,Tournament,Stage,Match Type,Match Name,Map,Team A,Eco_Round_Rating_A,Team_A_win
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,KRÜ Esports,0.238358,1.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,KRÜ Esports,0.112415,1.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Pearl,KRÜ Esports,-0.471407,0.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Split,KRÜ Esports,0.563010,1.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Lower Final,Cloud9 vs Leviatán,Bind,Cloud9,-0.120491,0.0
...,...,...,...,...,...,...,...,...
766,Valorant Champions 2023,Playoffs,Upper Semifinals,DRX vs Evil Geniuses,Fracture,DRX,-0.049698,0.0
767,Valorant Champions 2023,Playoffs,Upper Semifinals,DRX vs Evil Geniuses,Lotus,DRX,-0.331516,0.0
768,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Lotus,LOUD,-0.038252,0.0
769,Valorant Champions 2023,Playoffs,Upper Semifinals,LOUD vs Paper Rex,Pearl,LOUD,-0.049159,0.0


In [112]:
my_eco_rating_df = pd.concat([grouped_eco_rating_df_A.rename(columns={"Team A":"Team", "Eco_Round_Rating_A": "My_Eco_Rating", "Team_A_win":"Won Match"}), \
                    grouped_eco_rating_df_B.rename(columns={"Team B":"Team", "Eco_Round_Rating_B": "My_Eco_Rating", "Team_B_win":"Won Match"})], axis=0)

In [113]:
my_eco_rating_df
my_eco_rating_non_finals = my_eco_rating_df.loc[my_eco_rating_df["Tournament"]!="Valorant Champions 2023"]
#len(my_eco_rating_non_finals)/2 #=688
my_eco_rating_non_finals

,Tournament,Stage,Match Type,Match Name,Map,Team,My_Eco_Rating,Won Match
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,KRÜ Esports,0.238358,1.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Lotus,KRÜ Esports,0.112415,1.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Pearl,KRÜ Esports,-0.471407,0.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Split,KRÜ Esports,0.563010,1.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Lower Final,Cloud9 vs Leviatán,Bind,Cloud9,-0.120491,0.0
...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,DRX vs Team Secret,Bind,Team Secret,-0.339042,0.0
684,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,DRX vs Team Secret,Haven,Team Secret,0.362706,1.0
685,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,DRX vs Team Secret,Split,Team Secret,-0.345000,0.0
686,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,Paper Rex vs T1,Fracture,T1,-0.130697,0.0


In [114]:
1376/2

688.0

In [115]:


my_eco_rating_df_ab =\
      Eco_rounds_ab_2023.loc[Eco_rounds_ab_2023["Round Number"]==1]\
      [["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team B"]]\
            .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  .join(grouped_eco_rating_df_A\
                        .rename(columns={"Eco_Round_Rating_A":"Team_A_My_Eco_Rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  )\
                  .reset_index()\
                  .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  .join(grouped_eco_rating_df_B\
                        .rename(columns={"Eco_Round_Rating_B":"Team_B_My_Eco_Rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  )\
                  .reset_index()

my_eco_rating_non_finals_ab = my_eco_rating_df_ab.loc[my_eco_rating_df_ab["Tournament"]!="Valorant Champions 2023"]
#len(my_eco_rating_non_finals)/2 #=688
#my_eco_rating_non_finals_ab.isna().sum() #58


my_eco_rating_non_finals_ab = my_eco_rating_non_finals_ab.reset_index().drop(columns="index")
my_eco_rating_non_finals_ab

,Tournament,Stage,Match Type,Match Name,Map,Team B,Team A,Team_A_My_Eco_Rating,Team_A_win,Team_B_My_Eco_Rating,Team_B_win
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,MIBR,-0.244332,0.0,0.244332,1.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,KRÜ Esports,MIBR,-0.150451,0.0,0.150451,1.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,KRÜ Esports,FURIA,0.053344,1.0,-0.053344,0.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,KRÜ Esports,FURIA,-0.831858,0.0,0.831858,1.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,KRÜ Esports,FURIA,0.221960,0.0,-0.221960,1.0
...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,FNATIC,LOUD,-0.242662,0.0,0.242662,1.0
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,FNATIC,LOUD,-0.000640,0.0,0.000640,1.0
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,FNATIC,LOUD,0.132743,1.0,-0.132743,0.0
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,FNATIC,LOUD,0.166363,1.0,-0.166363,0.0


Next, we will consider the loadout values in first (MaxRounds=13) rounds. We will do a logistic regression on loadout values of both teams. That will give us probabilities of win. This will be our rating.

In [116]:
#Consider only the pre-final matches
Eco_rounds_ab_2023_non_finals = Eco_rounds_ab_2023.loc[Eco_rounds_ab_2023["Tournament"]!="Valorant Champions 2023"]
Eco_rounds_ab_2023_non_finals = Eco_rounds_ab_2023_non_finals.drop(columns=["Outcome_A", "Outcome_B", "Eco_Round_Rating_A", "Eco_Round_Rating_B"])
Eco_rounds_ab_2023_non_finals

,Tournament,Stage,Match Type,Match Name,Map,Team B,Round Number,Team A,Team_A_win,Team_B_win,Loadout_A,Loadout_B
1805,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,1,MIBR,False,True,3900,3800
1806,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,2,MIBR,False,True,4300,16200
1807,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,3,MIBR,False,True,21500,13900
1808,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,4,MIBR,False,True,7500,22600
1809,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,5,MIBR,False,True,22600,23600
...,...,...,...,...,...,...,...,...,...,...,...,...
16511,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,FNATIC,22,LOUD,False,True,21400,23200
16512,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,FNATIC,23,LOUD,False,True,9600,25000
16513,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,FNATIC,24,LOUD,False,True,22000,23200
16514,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Icebox,FNATIC,25,LOUD,False,True,23900,22600


In [117]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team A', 'Team B']


eco_rounds_13_team_ab_2023 = team_ab_2023
for i in range(13):
    eco_rounds_13_team_ab_2023 =\
    pd.merge(
        eco_rounds_13_team_ab_2023
        ,Eco_rounds_ab_2023_non_finals[Eco_rounds_ab_2023_non_finals["Round Number"] == i+1].drop(columns=["Round Number", "Team_A_win", "Team_B_win"])\
        ,on=keys
    )\
    .rename(columns={"Loadout_A":f"R{i+1}_A_lo","Loadout_B":f"R{i+1}_B_lo"})#\
    #.reset_index()    
# team_ab_2022

# for i in range(1):
#     eco_rounds_13_team_ab_2023 =\
#     pd.merge(
#         Eco_rounds_ab_2023_non_finals.rename(columns={"Team A":"Team"})\
#         ,Eco_rounds_ab_2023_non_finals[Ec["Round Number"] == i+1].drop(columns=["Round Number"])\
#         ,on=keys
#     )\
#     .rename(columns={"Loadout Value":f"R{i+1}_A_lo"})\
#     .rename(columns={"Team":"Team A", "Team B":"Team"})\
#     .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team'])\
#     .join(eco_rounds_for_prob[eco_rounds_for_prob["Round Number"] == i+1].drop(columns=["Round Number"])\
#         .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team'])\
#     )\
#     .reset_index()\
#     .rename(columns={"Loadout Value":f"R{i+1}_B_lo"})\
#     .rename(columns={"Team":"Team B"})\



In [118]:
eco_rounds13_team_ab_2023 = pd.merge(
        eco_rounds_13_team_ab_2023
        ,Eco_rounds_ab_2023_non_finals[Eco_rounds_ab_2023_non_finals["Round Number"] == 1].drop(columns=["Round Number", "Loadout_A", "Loadout_B"])\
        ,on=keys
    )
eco_rounds13_team_ab_2023

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team B,R1_A_lo,R1_B_lo,R2_A_lo,...,R10_A_lo,R10_B_lo,R11_A_lo,R11_B_lo,R12_A_lo,R12_B_lo,R13_A_lo,R13_B_lo,Team_A_win,Team_B_win
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,MIBR,KRÜ Esports,3900,3800,4300,...,23900,22800,11700,23100,20800,23900,3800,3800,False,True
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,MIBR,KRÜ Esports,3400,4100,2800,...,19700,22900,5800,23100,19900,24000,4300,3300,False,True
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,FURIA,KRÜ Esports,4000,3700,16000,...,22400,23000,20900,25100,23600,24000,4100,3800,True,False
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,FURIA,KRÜ Esports,3600,3900,15000,...,10100,23000,22200,24300,14400,24500,3700,3600,False,True
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,FURIA,KRÜ Esports,3800,3800,1500,...,11100,22700,23100,14400,24600,21900,4200,3800,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,LOUD,FNATIC,4200,3200,4200,...,23100,25000,23400,22500,24300,23200,3700,3500,False,True
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,LOUD,FNATIC,3600,3900,2200,...,24000,13700,24700,20000,25300,19300,3700,3600,False,True
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,LOUD,FNATIC,4100,3800,3000,...,22600,12500,22900,19500,21700,22700,3800,3900,True,False
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,LOUD,FNATIC,4200,3600,4500,...,23300,18000,25000,21000,23300,20200,4000,3600,True,False


In [119]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [120]:
Model1_X_features = ["R1_A_lo"   #Model 1 is log reg over loadout values of both teams in the first half
    ,"R1_B_lo"
    ,"R2_A_lo"
    ,"R2_B_lo"
    ,"R3_A_lo"
    ,"R3_B_lo"
    ,"R4_A_lo"
    ,"R4_B_lo"
    ,"R5_A_lo"
    ,"R5_B_lo"
    ,"R6_A_lo"
    ,"R6_B_lo"
    ,"R7_A_lo"
    ,"R7_B_lo"
    ,"R8_A_lo"
    ,"R8_B_lo"
    ,"R9_A_lo"
    ,"R9_B_lo"
    ,"R10_A_lo"
    ,"R10_B_lo"
    ,"R11_A_lo"
    ,"R11_B_lo"
    ,"R12_A_lo"
    ,"R12_B_lo"
    ,"R13_A_lo"
    ,"R13_B_lo"
    ]

In [121]:
M1_X = eco_rounds13_team_ab_2023[Model1_X_features]
M1_y_aw = eco_rounds13_team_ab_2023["Team_A_win"]
M1_y_bw = eco_rounds13_team_ab_2023["Team_B_win"]

In [122]:
lgr_aw = LogisticRegression()
lgr_bw = LogisticRegression()

In [123]:
lgr_aw.fit(M1_X, M1_y_aw)
lgr_aw_pred = lgr_aw.predict(M1_X)
lgr_aw_acc = accuracy_score(M1_y_aw, lgr_aw_pred)

lgr_bw.fit(M1_X, M1_y_bw)
lgr_bw_pred = lgr_bw.predict(M1_X)
lgr_bw_acc = accuracy_score(M1_y_bw, lgr_bw_pred)


c:\Users\parab\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\parab\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [124]:
print("lgr_aw accuracy:", lgr_aw_acc)
print("lgr_bw accuracy:", lgr_bw_acc)
print("mean of lgr_aw_pred: {}".format(np.mean(lgr_aw_pred)))
print("mean of aw_test: {}".format(np.mean(M1_y_aw)))
print("mean of lgr_bw_pred: {}".format(np.mean(lgr_bw_pred)))
print("mean of bw_test: {}".format(np.mean(M1_y_bw)))
print("coefficients:", lgr_aw.coef_)
lgr_aw.predict_proba(M1_X)


lgr_aw accuracy: 0.7558139534883721
lgr_bw accuracy: 0.7558139534883721
mean of lgr_aw_pred: 0.5203488372093024
mean of aw_test: 0.5087209302325582
mean of lgr_bw_pred: 0.4796511627906977
mean of bw_test: 0.49127906976744184
coefficients: [[-9.18720883e-05  3.90882487e-05 -1.23952559e-05 -8.54641307e-05
   6.79005390e-06 -2.24723221e-05  2.94049325e-05 -7.01508035e-07
   1.86458578e-05 -3.60397293e-05  2.45511290e-05 -7.24534117e-05
  -6.94755189e-06 -1.95213185e-05  3.43506548e-05 -3.96553046e-05
   6.33237374e-05 -1.87009202e-05  1.58834436e-05 -1.25400034e-05
   5.40820622e-05 -2.81915782e-05  6.09998819e-05 -9.61797009e-05
  -2.78970193e-05  5.79901504e-04]]


array([[0.91840147, 0.08159853],
       [0.9385669 , 0.0614331 ],
       [0.28043039, 0.71956961],
       ...,
       [0.44179268, 0.55820732],
       [0.20237348, 0.79762652],
       [0.12971111, 0.87028889]])

In [125]:
lgr_bw.predict_proba(M1_X).shape

(688, 2)

In [126]:
eco_rounds13_team_ab_2023["Team_A_Prob_Eco_Rating"] = lgr_aw.predict_proba(M1_X)[:,1]
eco_rounds13_team_ab_2023["Team_B_Prob_Eco_Rating"] = lgr_aw.predict_proba(M1_X)[:,0]
eco_rounds13_team_ab_2023


,Tournament,Stage,Match Type,Match Name,Map,Team A,Team B,R1_A_lo,R1_B_lo,R2_A_lo,...,R11_A_lo,R11_B_lo,R12_A_lo,R12_B_lo,R13_A_lo,R13_B_lo,Team_A_win,Team_B_win,Team_A_Prob_Eco_Rating,Team_B_Prob_Eco_Rating
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,MIBR,KRÜ Esports,3900,3800,4300,...,11700,23100,20800,23900,3800,3800,False,True,0.081599,0.918401
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,MIBR,KRÜ Esports,3400,4100,2800,...,5800,23100,19900,24000,4300,3300,False,True,0.061433,0.938567
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,FURIA,KRÜ Esports,4000,3700,16000,...,20900,25100,23600,24000,4100,3800,True,False,0.719570,0.280430
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,FURIA,KRÜ Esports,3600,3900,15000,...,22200,24300,14400,24500,3700,3600,False,True,0.229239,0.770761
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,FURIA,KRÜ Esports,3800,3800,1500,...,23100,14400,24600,21900,4200,3800,False,True,0.430492,0.569508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,LOUD,FNATIC,4200,3200,4200,...,23400,22500,24300,23200,3700,3500,False,True,0.245598,0.754402
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,LOUD,FNATIC,3600,3900,2200,...,24700,20000,25300,19300,3700,3600,False,True,0.407382,0.592618
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,LOUD,FNATIC,4100,3800,3000,...,22900,19500,21700,22700,3800,3900,True,False,0.558207,0.441793
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,LOUD,FNATIC,4200,3600,4500,...,25000,21000,23300,20200,4000,3600,True,False,0.797627,0.202373


In [127]:
my_eco_rating_non_finals_ab

,Tournament,Stage,Match Type,Match Name,Map,Team B,Team A,Team_A_My_Eco_Rating,Team_A_win,Team_B_My_Eco_Rating,Team_B_win
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,MIBR,-0.244332,0.0,0.244332,1.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,KRÜ Esports,MIBR,-0.150451,0.0,0.150451,1.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,KRÜ Esports,FURIA,0.053344,1.0,-0.053344,0.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,KRÜ Esports,FURIA,-0.831858,0.0,0.831858,1.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,KRÜ Esports,FURIA,0.221960,0.0,-0.221960,1.0
...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,FNATIC,LOUD,-0.242662,0.0,0.242662,1.0
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,FNATIC,LOUD,-0.000640,0.0,0.000640,1.0
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,FNATIC,LOUD,0.132743,1.0,-0.132743,0.0
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,FNATIC,LOUD,0.166363,1.0,-0.166363,0.0


In [128]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team A', 'Team B']
both_eco_ratings_non_finals = my_eco_rating_non_finals_ab\
        .drop(columns=["Team_A_win", "Team_B_win"] ).merge(eco_rounds13_team_ab_2023, on = keys)

In [129]:
both_eco_ratings_non_finals

,Tournament,Stage,Match Type,Match Name,Map,Team B,Team A,Team_A_My_Eco_Rating,Team_B_My_Eco_Rating,R1_A_lo,...,R11_A_lo,R11_B_lo,R12_A_lo,R12_B_lo,R13_A_lo,R13_B_lo,Team_A_win,Team_B_win,Team_A_Prob_Eco_Rating,Team_B_Prob_Eco_Rating
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Bind,KRÜ Esports,MIBR,-0.244332,0.244332,3900,...,11700,23100,20800,23900,3800,3800,False,True,0.081599,0.918401
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Round 1,MIBR vs KRÜ Esports,Lotus,KRÜ Esports,MIBR,-0.150451,0.150451,3400,...,5800,23100,19900,24000,4300,3300,False,True,0.061433,0.938567
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Split,KRÜ Esports,FURIA,0.053344,-0.053344,4000,...,20900,25100,23600,24000,4100,3800,True,False,0.719570,0.280430
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Ascent,KRÜ Esports,FURIA,-0.831858,0.831858,3600,...,22200,24300,14400,24500,3700,3600,False,True,0.229239,0.770761
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Upper Quarterfinals,FURIA vs KRÜ Esports,Bind,KRÜ Esports,FURIA,0.221960,-0.221960,3800,...,23100,14400,24600,21900,4200,3800,False,True,0.430492,0.569508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Ascent,FNATIC,LOUD,-0.242662,0.242662,4200,...,23400,22500,24300,23200,3700,3500,False,True,0.245598,0.754402
684,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Fracture,FNATIC,LOUD,-0.000640,0.000640,3600,...,24700,20000,25300,19300,3700,3600,False,True,0.407382,0.592618
685,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Split,FNATIC,LOUD,0.132743,-0.132743,4100,...,22900,19500,21700,22700,3800,3900,True,False,0.558207,0.441793
686,Champions Tour 2023: Lock-In Sao Paulo,Playoffs,Grand Final,LOUD vs FNATIC,Lotus,FNATIC,LOUD,0.166363,-0.166363,4200,...,25000,21000,23300,20200,4000,3600,True,False,0.797627,0.202373


In [130]:
# Save a dataframe with "My_Eco_Rating" with the formula, "Prob_Eco_Rating" with probability as rating, and all rounds loadout values  
both_eco_ratings_non_finals.to_csv("../data/vct_2023/eco_ratings.csv")